In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Variables
r_ball_m = 0.1066       # [m]
m_ball_kg = 0.28        # [kg]
rho_h20_kgpm3 = 997     # [kg/m^3]
coeff_drag = 0.5        # [unitless]

# Constants
g = 9.81    # [m/s^2]

# Derived values
cross_section_area_ball_m2 = np.pi*np.power(r_ball_m, 2)
volume_ball_m3 = (4/3)*np.pi*np.power(r_ball_m, 3)

### Forces

In [3]:
# Constant forces 
F_buoyancy_N = rho_h20_kgpm3 * g * volume_ball_m3
F_g_N = m_ball_kg * g

In [ ]:
F_drag_N = (1/2) * rho_h20_kgpm3 * np.power(v_ball_mps, 2) * coeff_drag * cross_section_area_ball_m2

## Determining behavior at terminal velocity

In [10]:
# Determining terminal velocity of ball underwater 
v_ball_mps_numerator = 2*(F_buoyancy_N - F_g_N)
v_ball_mps_denominator = rho_h20_kgpm3*coeff_drag*np.pi*np.power(r_ball_m, 2)
v_ball_mps = np.sqrt(v_ball_mps_numerator/v_ball_mps_denominator)
v_ball_mps_string = "{:.3f}".format(v_ball_mps)

print("Terminal velocity of ball: " + v_ball_mps_string + " m/s")

Terminal velocity of ball: 2.295 m/s
